# Parallel high-performance stencil computations on xPUs

Ludovic Räss & Ivan Utkin

_ETH Zurich_

![gpu](./figures/logo2.png)

## The nice to have features

Wouldn't it be nice to have single code that:
- runs both on CPUs and GPUs (xPUs)?
- one can use for prototyping and production?
- runs at optimal performance?

Hold on 🙂 ...